In [1]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd
pd.set_option('display.max_rows', 40)

from vivarium_research_prl.noise import corruption, fake_names, noisify

!date
!whoami
!uname -a
!pwd

Wed 22 Feb 2023 03:02:00 PM PST
ndbs
Linux int-slurm-sarchive-p0001 5.4.0-135-generic #152-Ubuntu SMP Wed Nov 23 20:19:22 UTC 2022 x86_64 x86_64 x86_64 GNU/Linux
/mnt/share/code/ndbs/vivarium_research_prl/noise


In [2]:
%load_ext autoreload
%autoreload 2

# Goal: Enhance the `random_choice` function to allow excluding the current choice

# Create random generator

In [3]:
rng = np.random.default_rng(983292329775)

# Create Series for testing and test code for implementation

In [4]:
s = pd.Series(list('abdciaalgkaghha'))
s

0     a
1     b
2     d
3     c
4     i
5     a
6     a
7     l
8     g
9     k
10    a
11    g
12    h
13    h
14    a
dtype: object

In [5]:
sc = s.copy()
sc.loc[s.index]

0     a
1     b
2     d
3     c
4     i
5     a
6     a
7     l
8     g
9     k
10    a
11    g
12    h
13    h
14    a
dtype: object

In [6]:
a = rng.choice(list('abc'), len(s))
a

array(['a', 'b', 'b', 'a', 'b', 'a', 'a', 'b', 'b', 'a', 'b', 'a', 'b',
       'b', 'c'], dtype='<U1')

In [7]:
a == s

0      True
1      True
2     False
3     False
4     False
5      True
6      True
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
dtype: bool

In [8]:
type(a==s)

pandas.core.series.Series

In [9]:
u = (a==s)
s[u]

0    a
1    b
5    a
6    a
dtype: object

In [10]:
a[u]

array(['a', 'b', 'a', 'a'], dtype='<U1')

In [11]:
s[~u]

2     d
3     c
4     i
7     l
8     g
9     k
10    a
11    g
12    h
13    h
14    a
dtype: object

In [12]:
a[~u]

array(['b', 'a', 'b', 'b', 'b', 'a', 'b', 'a', 'b', 'b', 'c'], dtype='<U1')

# Test 1st algorithm (resampling)

In [13]:
rc = corruption.random_choice(s, list('ab'), random_state=54321)
rc

0     a
1     b
2     b
3     b
4     b
5     a
6     b
7     a
8     a
9     a
10    b
11    a
12    b
13    b
14    b
dtype: object

In [14]:
s == rc

0      True
1      True
2     False
3     False
4     False
5      True
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
dtype: bool

In [15]:
rc2 = corruption.random_choice(s, list('ab'), exclude_current=True, random_state=54321)
rc2

0     b
1     a
2     b
3     b
4     b
5     b
6     b
7     a
8     a
9     a
10    b
11    a
12    b
13    b
14    b
dtype: object

In [16]:
s == rc2

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
dtype: bool

In [17]:
s2 = pd.Series(rng.choice(['M', 'F'], 15))
s2

0     F
1     F
2     F
3     M
4     M
5     M
6     M
7     M
8     M
9     M
10    M
11    M
12    M
13    F
14    M
dtype: object

In [18]:
t2 = corruption.random_choice(s2, ['M', 'F'], random_state=85737)
t2

0     M
1     F
2     M
3     M
4     M
5     F
6     F
7     F
8     F
9     M
10    F
11    M
12    M
13    F
14    M
dtype: object

In [19]:
s2==t2

0     False
1      True
2     False
3      True
4      True
5     False
6     False
7     False
8     False
9      True
10    False
11     True
12     True
13     True
14     True
dtype: bool

In [20]:
(s2==t2).sum()

8

In [21]:
t2b = corruption.random_choice(s2, ['M', 'F'], exclude_current=True, random_state=85737)
t2b

0     M
1     M
2     M
3     F
4     F
5     F
6     F
7     F
8     F
9     F
10    F
11    F
12    F
13    M
14    F
dtype: object

In [22]:
s2==t2b

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
dtype: bool

# Test again with larger data

In [23]:
size = int(1e6)
s3 = pd.Series(rng.choice(list('abcdef'), size))
s3

0         e
1         a
2         c
3         a
4         e
         ..
999995    f
999996    b
999997    c
999998    f
999999    d
Length: 1000000, dtype: object

In [24]:
t3a = corruption.random_choice(s3, s3.unique(), random_state=rng)
t3a

0         b
1         b
2         e
3         e
4         e
         ..
999995    c
999996    e
999997    e
999998    f
999999    f
Length: 1000000, dtype: object

In [25]:
same3a = (t3a == s3)
print(same3a.sum(), same3a.sum()/size)

167072 0.167072


In [26]:
t3b = corruption.random_choice(s3, s3.unique(), exclude_current=True, random_state=rng)
t3b

0         a
1         b
2         a
3         e
4         b
         ..
999995    b
999996    f
999997    b
999998    e
999999    c
Length: 1000000, dtype: object

In [27]:
same3b = (t3b == s3)
print(same3b.sum(), same3b.sum()/size)

0 0.0


# Test 2nd algorithm (explicit exclusion using arrays)

In [28]:
np.full(5, 7)

array([7, 7, 7, 7, 7])

In [29]:
val = 'b'
choices = list('abc')
p = [1/2, 1/3, 1/6]
choices, p = map(np.asarray, (choices, p))
np.where(choices != val, p, 0)

array([0.5       , 0.        , 0.16666667])

In [30]:
p_cond = np.where(choices != val, p, 0)
p_cond /= p_cond.sum()
p_cond

array([0.75, 0.  , 0.25])

In [31]:
for _ in range(10):
    print(corruption.random_choice2(6, [1,3,6]), sep=' ', end=' ')
print()
for _ in range(10):
    print(corruption.random_choice2(6, [1,3,6], exclude_current=True), sep=' ', end=' ')

1 1 3 1 3 3 3 6 3 1 
1 1 3 3 1 1 3 1 3 3 

In [32]:
sixes = pd.Series(6, index=range(10))
corruption.random_choice2(sixes, [1,3,6], random_state=8888)

0    1
1    1
2    3
3    1
4    3
5    6
6    3
7    6
8    6
9    3
dtype: int64

In [33]:
corruption.random_choice2(sixes, [1,3,6], exclude_current=True, random_state=8888)

0    1
1    1
2    3
3    3
4    1
5    1
6    1
7    1
8    3
9    1
dtype: int64

In [34]:
x = pd.Series(s, index=range(14,0,-1))
x

14    a
13    h
12    h
11    g
10    a
9     k
8     g
7     l
6     a
5     a
4     i
3     c
2     d
1     b
dtype: object

In [35]:
s

0     a
1     b
2     d
3     c
4     i
5     a
6     a
7     l
8     g
9     k
10    a
11    g
12    h
13    h
14    a
dtype: object

In [36]:
s.to_numpy()

array(['a', 'b', 'd', 'c', 'i', 'a', 'a', 'l', 'g', 'k', 'a', 'g', 'h',
       'h', 'a'], dtype=object)

In [37]:
i = pd.Series(rng.choice([True, False], len(t2)), index=t2)
i

M    False
F    False
M    False
M    False
M    False
F    False
F     True
F    False
F     True
M     True
F     True
M     True
M    False
F    False
M    False
dtype: bool

In [38]:
s.to_numpy()[i]

array(['a', 'g', 'k', 'a', 'g'], dtype=object)

In [39]:
s.dtypes

dtype('O')

In [40]:
s.dtype

dtype('O')

# Test after refactoring logic into different order

In [41]:
for _ in range(10):
    print(corruption.random_choice(3, [1,3,6]), sep=' ', end=' ')
print()
for _ in range(10):
    print(corruption.random_choice(3, [1,3,6], exclude_current=True), sep=' ', end=' ')

6 6 6 6 1 3 1 6 6 6 
6 1 6 1 1 1 6 6 1 1 

# Now refactor to write a single function that calls one of the two algorithms as a subroutine

Copy current code, then modify it

In [42]:
def random_choice(
    current_choice,
    choices,
    exclude_current=True,
    exclusion_algorithm='explicit_exclusion', # Ignored if exclude_current is False
    replace=True,
    p=None,
    shuffle=True,
    random_state=None,
):
    if not exclude_current:
        rng = np.random.default_rng(random_state)
        is_series = isinstance(current_choice, pd.Series)
        if is_series:
            new_choice = pd.Series(
                rng.choice(choices, len(current_choice), replace, p, shuffle=shuffle),
                index=current_choice.index, name=current_choice.name)
        else:
            # Set shape=None not shape=1 so that rng.choice returns a scalar not an array
            new_choice = rng.choice(choices, None, replace, p, shuffle=shuffle)
    elif exclusion_algorithm == 'explicit_exclusion':
        new_choice = random_different_choice_via_explicit_exclusion(
            current_choice, choices, replace, p, shuffle, random_state)
    elif exclusion_algorithm == 'resampling':
        new_choice = random_different_choice_via_resampling(
            current_choice, choices, replace, p, shuffle, random_state)
    else:
        raise ValueError(f'Unrecognized exclusion algorithm: {exclusion_algorithm}')
    return new_choice

def random_different_choice_via_explicit_exclusion(
    current_choice,
    choices,
    replace=True,
    p=None,
    shuffle=True,
    random_state=None,
):
    rng = np.random.default_rng(random_state)
    is_series = isinstance(current_choice, pd.Series)
    choices = np.asarray(choices)
    if p is None:
        p = np.full(len(choices), 1/len(choices))
    if is_series:
        new_choice = pd.Series(
            np.empty(len(current_choice), dtype=current_choice.dtype),
            index=current_choice.index, name=current_choice.name)
        current_not_in_choices = ~current_choice.isin(choices)
        num_rows = current_not_in_choices.sum()
        new_choice[current_not_in_choices] = rng.choice(
                choices, num_rows, replace, p, shuffle=shuffle)
        for c in choices:
            p_cond = np.where(choices != c, p, 0)
            p_cond /= p_cond.sum()
            current_equals_c = (current_choice == c)
            num_rows = current_equals_c.sum()
            new_choice[current_equals_c] = rng.choice(
                choices, num_rows, replace, p_cond, shuffle=shuffle)
    else: # Scalar version
        p_cond = np.where(choices != current_choice, p, 0)
        p_cond /= p_cond.sum()
        new_choice = rng.choice(choices, None, replace, p_cond, shuffle=shuffle)

    return new_choice

def random_different_choice_via_resampling(
    current_choice,
    choices,
    replace=True,
    p=None,
    shuffle=True,
    random_state=None,
):
    rng = np.random.default_rng(random_state)
    is_series = isinstance(current_choice, pd.Series)
    if is_series:
        shape = len(current_choice)
#         new_choice = current_choice.copy()
        new_choice = pd.Series(
            np.empty(shape, dtype=current_choice.dtype),
            index=current_choice.index, name=current_choice.name)
        # No values have changed until the loop executes the first time
        unchanged = pd.Series(True, index=new_choice.index)
    else:
        # Set shape=None not shape=1 so that rng.choice returns a scalar not an array
        shape = None
    # Resample until all values differ from current
    done = False
    while not done:
        random_choice = rng.choice(choices, shape, replace, p, shuffle=shuffle)
        if is_series:
            new_choice[unchanged] = random_choice
            unchanged = (new_choice == current_choice)
            shape = unchanged.sum()
            done = (shape == 0)
        else: # Scalar version
            new_choice = random_choice
            done = (new_choice != current_choice)
    return new_choice


In [43]:
np.empty(5, dtype=int)

array([94497817845504,              0, 94497804431264, 94497811650416,
                   48])

In [44]:
pd.Series(np.empty(5, dtype=int))

0    94497818264016
1                 0
2                 0
3                 0
4        4294967306
dtype: int64

In [45]:
s

0     a
1     b
2     d
3     c
4     i
5     a
6     a
7     l
8     g
9     k
10    a
11    g
12    h
13    h
14    a
dtype: object

In [46]:
random_choice(s, ['a', 'b'], random_state=9999)

0     b
1     a
2     b
3     b
4     a
5     b
6     b
7     b
8     a
9     b
10    b
11    b
12    b
13    a
14    b
dtype: object

In [47]:
random_choice('a', ['a', 'b'], random_state=9999)

'b'

In [48]:
random_choice('c', ['a', 'b'])

'a'

In [49]:
random_choice('c', ['c'])

/tmp/ipykernel_931384/3198667531.py:61: RuntimeWarning: invalid value encountered in divide
  p_cond /= p_cond.sum()


ValueError: probabilities contain NaN

In [50]:
random_choice(s, ['b'], random_state=9999)

/tmp/ipykernel_931384/3198667531.py:54: RuntimeWarning: invalid value encountered in divide
  p_cond /= p_cond.sum()


ValueError: probabilities contain NaN

# Test after incorporating above code into corruption module

## Test both algorithms and test when `exclude_current` is False

In [86]:
s3.unique()

array(['e', 'a', 'c', 'd', 'b', 'f'], dtype=object)

In [87]:
t3c = corruption.random_choice(s3, list('abcd'), random_state=88888)
t3c.unique()

array(['c', 'a', 'd', 'b'], dtype=object)

In [88]:
(t3c == s3).any()

False

In [89]:
t3d = corruption.random_choice(s3, list('abcd'), False, random_state=88888)
t3d

0         c
1         c
2         a
3         d
4         b
         ..
999995    b
999996    b
999997    c
999998    b
999999    b
Length: 1000000, dtype: object

In [90]:
(t3d == s3).sum()

166067

In [91]:
t3e = corruption.random_choice(
    s3, list('abcd'), exclusion_algorithm='resampling', random_state=88888)
t3e

0         c
1         c
2         a
3         d
4         b
         ..
999995    b
999996    d
999997    d
999998    b
999999    b
Length: 1000000, dtype: object

In [92]:
t3e.unique()

array(['c', 'a', 'd', 'b'], dtype=object)

In [93]:
(t3e == s3).any()

False

## Time the two exclusion algorithms

Explicit exclusion is faster than resampling by a factor of more than two for this particular test data, and is slower than _not_ excluding the current choice by a factor of 20.

In [94]:
%timeit corruption.random_choice(s3, list('abcd'))

266 ms ± 1.75 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [97]:
%timeit corruption.random_choice(s3, list('abcd'), exclusion_algorithm='resampling')

623 ms ± 65.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [98]:
%timeit corruption.random_choice(s3, list('abcd'), exclusion_algorithm=None)

13.5 ms ± 45.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Make sure `miswrite_age` still works since I had to edit the call to `random_choice` for this function

Including -2 in the increments should cause some elements where age = 1 to remain the same after noising.

In [99]:
age = pd.Series(rng.choice(100, size))
age
age_noisy = corruption.miswrite_age(age, [-2, -1, 1, 2])
age_noisy

0         18
1         32
2         82
3         43
4         19
          ..
999995    48
999996    95
999997    22
999998    44
999999    23
Length: 1000000, dtype: int64

In [100]:
(age == age_noisy).sum()

2526

In [101]:
age.loc[(age == age_noisy)].unique()

array([1])

In [102]:
(age - age_noisy).unique()

array([-1,  1,  2, -2,  0])